In [1]:
from llvmlite import ir

# Create some useful types
intty = ir.IntType(32)
fnty = ir.FunctionType(intty, (intty, intty))

# Create an empty module...
module = ir.Module(name="name")
# and declare a function named "fpadd" inside it
func = ir.Function(module, fnty, name="fpadd")

# Now implement the function
block = func.append_basic_block(name="entry")


added = ir.Instruction(block, intty, 'add', [
    func.args[0],
    func.args[1]
])
block.instructions = [
    added,
    ir.Ret(block, 'ret', added)
]

# Print the module IR
print(module)

; ModuleID = "name"
target triple = "unknown-unknown-unknown"
target datalayout = ""

define i32 @"fpadd"(i32 %".1", i32 %".2") 
{
entry:
  %".4" = add i32 %".1", %".2"
  ret i32 %".4"
}



In [9]:
from llvmlite import binding as llvm

In [10]:
def execute(ir_mod):
    llvm.initialize()
    llvm.initialize_native_target()
    llvm.initialize_native_asmprinter()

    llmod = llvm.parse_assembly(str(ir_mod))

    print('optimized'.center(80, '-'))
    pmb = llvm.create_pass_manager_builder()
    pmb.opt_level = 1
    pm = llvm.create_module_pass_manager()
    pmb.populate(pm)
    pm.run(llmod)
    print(llmod)

    target_machine = llvm.Target.from_default_triple().create_target_machine()

    with llvm.create_mcjit_compiler(llmod, target_machine) as ee:
        ee.finalize_object()
        cfptr = ee.get_function_address("fpadd")

        from ctypes import CFUNCTYPE, c_int

        cfunc = CFUNCTYPE(c_int, c_int, c_int)(cfptr)

        # TEST
        for i in range(12):
            res = cfunc(i, i)
            print('fib({}) = {}'.format(i, res))

        # Get CFG
        ll_fib_more = llmod.get_function('fpadd')
        cfg = llvm.get_function_cfg(ll_fib_more)
        llvm.view_dot_graph(cfg, view=True)

In [11]:
execute(module)

-----------------------------------optimized------------------------------------
; ModuleID = '<string>'
source_filename = "<string>"
target triple = "unknown-unknown-unknown"

; Function Attrs: norecurse nounwind readnone
define i32 @fpadd(i32 %.1, i32 %.2) local_unnamed_addr #0 {
entry:
  %.4 = add i32 %.2, %.1
  ret i32 %.4
}

attributes #0 = { norecurse nounwind readnone }

fib(0) = 0
fib(1) = 2
fib(2) = 4
fib(3) = 6
fib(4) = 8
fib(5) = 10
fib(6) = 12
fib(7) = 14
fib(8) = 16
fib(9) = 18
fib(10) = 20
fib(11) = 22


In [12]:
!pip install git+https://github.com/sklam/etude-tict.git

  Cloning https://github.com/sklam/etude-tict.git to /private/var/folders/m7/t8dvwtnn32z84333p845tly40000gn/T/pip-req-build-7mx62p4k
fatal: unable to access 'https://github.com/sklam/etude-tict.git/': Could not resolve host: github.com
Command "git clone -q https://github.com/sklam/etude-tict.git /private/var/folders/m7/t8dvwtnn32z84333p845tly40000gn/T/pip-req-build-7mx62p4k" failed with error code 128 in None


In [28]:
"""
Define a mini-language to compute the fibonnaci sequence.

Uses LLVM for codegeneration.
"""

import inspect
from llvmlite import ir
from llvmlite import binding as llvm

from kvbacker import (
    ResourceManager,
    Managed,
    ManagedList,
    graphviz_render_revisions,
)


class FunctionCollections(Managed):
    pass


class ContextResources(ResourceManager):
    pass


class Expr(Managed):
    pass


class FunctionDecl(Expr):
    def init(self, name):
        self.name = name

    def __call__(self, *args):
        args = self._resmngr.new(
            ArgListVal,
            [_fix_value(self._resmngr, x) for x in args],
        )
        return self._resmngr.new(CallVal, op=self, args=args)

    def codegen(self, cgstate):
        mod = cgstate.builder.module
        return mod.get_global(self.name)


class FunctionDefn(Expr):
    def init(self, name, expr, arity):
        self.name = name
        self.expr = expr
        self.arity = arity

    def codegen_definition(self, ir_mod):
        fn = self.codegen_declare(ir_mod)
        fn.calling_convention = 'fastcc'
        entry_block = fn.append_basic_block('entry')
        main_block = fn.append_basic_block('main')
        irbuilder = ir.IRBuilder()
        cgstate = CodegenState(irbuilder, entry_block)
        irbuilder.position_at_end(main_block)
        value = self.expr.codegen(cgstate)
        irbuilder.ret(value)
        # Clean up
        irbuilder.position_at_end(entry_block)
        irbuilder.branch(main_block)

    def codegen_declare(self, ir_mod):
        try:
            return ir_mod.get_global(self.name)
        except KeyError:
            argty = ir.IntType(32)
            fnty = ir.FunctionType(argty, [argty] * self.arity)
            fn = ir.Function(ir_mod, fnty, name=self.name)
            return fn


class ConstVal(Expr):
    def init(self, value):
        self.value = value

    def codegen(self, cgstate):
        intty = ir.IntType(32)
        return intty(self.value)


class ParamVal(Expr):
    def init(self, name, pos):
        self.name = name
        self.pos = pos

    def codegen(self, cgstate):
        builder = cgstate.builder
        fn = builder.function
        arg = fn.args[self.pos]
        return arg


class CallVal(Expr):
    def init(self, op, args):
        self.op = op
        self.args = args

    def codegen(self, cgstate):
        if isinstance(self.op, Expr):
            assert isinstance(self.op, FunctionDecl)
            callee = self.op.codegen(cgstate)
            args = [a.codegen(cgstate) for a in self.args]
            return cgstate.builder.call(callee, args)
        else:
            builder = cgstate.builder
            assert len(self.args) == 2
            lhs = self.args[0].codegen(cgstate)
            rhs = self.args[1].codegen(cgstate)
            if self.op == '+':
                res = builder.add(lhs, rhs)
            elif self.op == '-':
                res = builder.sub(lhs, rhs)
            elif self.op == '>':
                res = builder.icmp_signed('>', lhs, rhs)
            elif self.op == '==':
                res = builder.icmp_signed('==', lhs, rhs)
            else:
                raise NotImplementedError(self.op)
            return res


class ArgListVal(ManagedList):
    pass


class IfElseVal(Expr):
    def init(self, pred, then_expr, else_expr):
        self.pred = pred
        self.then_expr = then_expr
        self.else_expr = else_expr

    def codegen(self, cgstate):
        builder = cgstate.builder
        bb_then = builder.append_basic_block('then')
        bb_else = builder.append_basic_block('else')
        bb_after = builder.append_basic_block('endif')

        pred = self.pred.codegen(cgstate)
        builder.cbranch(pred, bb_then, bb_else)

        builder.position_at_end(cgstate.entry_block)
        phi = builder.alloca(ir.IntType(32))

        builder.position_at_end(bb_then)
        then_value = self.then_expr.codegen(cgstate)
        builder.store(then_value, phi)
        builder.branch(bb_after)

        builder.position_at_end(bb_else)
        else_value = self.else_expr.codegen(cgstate)
        builder.store(else_value, phi)
        builder.branch(bb_after)

        builder.position_at_end(bb_after)
        return builder.load(phi)


class Context:
    def __init__(self):
        self._rm = ContextResources()
        self._declfuncs = {}
        self._definitions = {}

    def define(self, fn):
        fname = fn.__name__
        fndecl = self._rm.new(FunctionDecl, name=fname)
        self._declfuncs[fname] = {'decl': fndecl, 'defn': fn}
        return fndecl

    def visualize(self):
        return self._rm.visualize()

    def codegen(self):
        ir_mod = ir.Module()
        for k, defn in self._definitions.items():
            defn.codegen_declare(ir_mod)
        for k, defn in self._definitions.items():
            defn.codegen_definition(ir_mod)
        return ir_mod

    def materialize(self):
        while self._declfuncs:
            name, info = self._declfuncs.popitem()
            defn = info['defn']
            defn = self._build_definition(name, defn)
            self._definitions[name] = defn

    def _build_definition(self, name, fn):
        sig = inspect.signature(fn)
        params = list(sig.parameters.items())
        kwargs = {params[0][0]: self}
        for i, (k, v) in enumerate(params[1:]):
            kwargs[k] = self._rm.new(ParamVal, name=k, pos=i)
        expr = fn(**kwargs)
        defn = self._rm.new(
            FunctionDefn, name=name, expr=expr, arity=len(params) - 1,
        )
        return defn

    def call(self, op, args):
        return self._rm.new(CallVal, op=op, args=self._arglist(*args))

    def ifelse(self, pred, then_expr, else_expr):
        return self._rm.new(
            IfElseVal,
            pred=pred,
            then_expr=self._fix_value(then_expr),
            else_expr=self._fix_value(else_expr),
        )

    def _arglist(self, *args):
        return self._rm.new(
            ArgListVal,
            values=[self._fix_value(x) for x in args],
        )

    def _fix_value(self, val):
        return _fix_value(self._rm, val)


def _fix_value(rm, val):
    if isinstance(val, Expr):
        return val
    else:
        return rm.new(ConstVal, value=val)


class CodegenState:
    def __init__(self, ir_builder, entry_block):
        self.builder = ir_builder
        self.entry_block = entry_block


def make_c_wrapper(fn_callee):
    mod = fn_callee.module
    fnty = fn_callee.function_type
    fn = ir.Function(mod, fnty, name='entry_' + fn_callee.name)
    builder = ir.IRBuilder(fn.append_basic_block())
    builder.ret(builder.call(fn_callee, fn.args))


def execute(ir_mod):
    llvm.initialize()
    llvm.initialize_native_target()
    llvm.initialize_native_asmprinter()

    llmod = llvm.parse_assembly(str(ir_mod))

    print('optimized'.center(80, '-'))
    pmb = llvm.create_pass_manager_builder()
    pmb.opt_level = 0
    pm = llvm.create_module_pass_manager()
    pmb.populate(pm)
    pm.run(llmod)
    print(llmod)

    target_machine = llvm.Target.from_default_triple().create_target_machine()

    with llvm.create_mcjit_compiler(llmod, target_machine) as ee:
        ee.finalize_object()
        cfptr = ee.get_function_address("entry_fib")

        from ctypes import CFUNCTYPE, c_int

        cfunc = CFUNCTYPE(c_int, c_int)(cfptr)

        # TEST
        for i in range(12):
            res = cfunc(i)
            print('fib({}) = {}'.format(i, res))

        # Get CFG
        ll_fib_more = llmod.get_function('fib_more')
        cfg = llvm.get_function_cfg(ll_fib_more)
        llvm.view_dot_graph(cfg, view=True)


def test():
    context = Context()

    @context.define
    def fib(ctx, n):
        return fib_more(n, 0, 1)

    @context.define
    def fib_more(ctx, n, a, b):
        pred_cont = ctx.call('>', [n, 1])
        minus1 = ctx.call('-', [n, 1])
        ab = ctx.call('+', [a, b])
        added = fib_more(minus1, b, ab)

        n_eq_1 = ctx.call('==', [n, 1])
        return ctx.ifelse(pred_cont, added,
                          ctx.ifelse(n_eq_1, b, a))

    context.materialize()

    ir_mod = context.codegen()
    context.visualize().render(view=True)
    make_c_wrapper(ir_mod.get_global('fib'))
    print(ir_mod)

    execute(ir_mod)

    rendered = graphviz_render_revisions(context._rm)
    return ir_mod
    with open('example_minilang.html', 'w') as fout:
        print(rendered, file=fout)


# if __name__ == '__main__':
#     test()


In [13]:
mod = test()

NameError: name 'test' is not defined

In [14]:
mod.globals

OrderedDict([('_ZN8__main__11loop_fn$241E5ArrayIxLi1E1C7mutable7alignedE',
              <ir.Function '_ZN8__main__11loop_fn$241E5ArrayIxLi1E1C7mutable7alignedE' of type 'i32 (i64*, {i8*, i32}**, i8*, i8*, i64, i64, i64*, i64, i64)*'>),
             ('_ZN7cpython8__main__11loop_fn$241E5ArrayIxLi1E1C7mutable7alignedE',
              <ir.Function '_ZN7cpython8__main__11loop_fn$241E5ArrayIxLi1E1C7mutable7alignedE' of type 'i8* (i8*, i8*, i8*)*'>),
             ('PyArg_UnpackTuple',
              <ir.Function 'PyArg_UnpackTuple' of type 'i32 (i8*, i8*, i64, i64, ...)*'>),
             ('.const.loop_fn',
              <ir.GlobalVariable '.const.loop_fn' of type '[8 x i8]*'>),
             ('_ZN08NumbaEnv8__main__11loop_fn$241E5ArrayIxLi1E1C7mutable7alignedE',
              <ir.GlobalVariable '_ZN08NumbaEnv8__main__11loop_fn$241E5ArrayIxLi1E1C7mutable7alignedE' of type 'i8**'>),
             ('PyErr_SetString',
              <ir.Function 'PyErr_SetString' of type 'void (i8*, i8*)*'>),
      

In [23]:
fib_more = mod.globals['fib_more']

In [24]:
fib_more??

Type:        Function
String form:
define fastcc i32 @"fib_more"(i32 %".1", i32 %".2", i32 %".3") 
{
entry:
  %".7" = alloca i32
  %".15" = alloca i32
  br label %"main"
main:
  %".5" = icmp sgt i32 %".1", 1
  br i1 %".5", label %"then", label %"else"
then:
  %".8" = sub i32 %".1", 1
  %".9" = add i32 %".2", %".3"
  %".10" = call fastcc i32 @"fib_more"(i32 %".8", i32 %".3", i32 %".9")
  store i32 %".10", i32* %".7"
  br label %"endif"
else:
  %".13" = icmp eq i32 %".1", 1
  br i1 %".13", label %"then.1", label %"else.1"
endif:
  %".23" = load i32, i32* %".7"
  ret i32 %".23"
then.1:
  store i32 %".3", i32* %".15"
  br label %"endif.1"
else.1:
  store i32 %".2", i32* %".15"
  br label %"endif.1"
endif.1:
  %".20" = load i32, i32* %".15"
  store i32 %".20", i32* %".7"
  br label %"endif"
}
File:        /usr/local/miniconda3/envs/metadsl/lib/python3.7/site-packages/llvmlite/ir/values.py
Source:     
class Function(GlobalValue, _HasMetadata):
    """Represent a LLVM Function but does uses 

In [25]:
fib_more.blocks

[<ir.Block 'entry' of type 'label'>,
 <ir.Block 'main' of type 'label'>,
 <ir.Block 'then' of type 'label'>,
 <ir.Block 'else' of type 'label'>,
 <ir.Block 'endif' of type 'label'>,
 <ir.Block 'then.1' of type 'label'>,
 <ir.Block 'else.1' of type 'label'>,
 <ir.Block 'endif.1' of type 'label'>]

In [32]:
then = fib_more.blocks[2]

In [33]:
then??

Type:        Block
String form:
%"then" = then:
  %".8" = sub i32 %".1", 1
  %".9" = add i32 %".2", %".3"
  %".10" = call fastcc i32 @"fib_more"(i32 %".8", i32 %".3", i32 %".9")
  store i32 %".10", i32* %".7"
  br label %"endif"
File:        /usr/local/miniconda3/envs/metadsl/lib/python3.7/site-packages/llvmlite/ir/values.py
Source:     
class Block(NamedValue):
    """
    A LLVM IR basic block. A basic block is a sequence of
    instructions whose execution always goes from start to end.  That
    is, a control flow instruction (branch) can only appear as the
    last instruction, and incoming branches can only jump to the first
    instruction.
    """

    def __init__(self, parent, name=''):
        super(Block, self).__init__(parent, types.LabelType(), name=name)
        self.scope = parent.scope
        self.instructions = []
        self.terminator = None

    @property
    def is_terminated(self):
        return self.terminator is not None

    @property
    def function(self)

In [34]:
then.instructions

[<ir.Instruction '.8' of type 'i32', opname 'sub', operands (<ir.Argument '.1' of type i32>, <ir.Constant type='i32' value=1>)>,
 <ir.Instruction '.9' of type 'i32', opname 'add', operands (<ir.Argument '.2' of type i32>, <ir.Argument '.3' of type i32>)>,
 <ir.CallInstr '.10' of type 'i32', opname 'call', operands [<ir.Function 'fib_more' of type 'i32 (i32, i32, i32)*'>, <ir.Instruction '.8' of type 'i32', opname 'sub', operands (<ir.Argument '.1' of type i32>, <ir.Constant type='i32' value=1>)>, <ir.Argument '.3' of type i32>, <ir.Instruction '.9' of type 'i32', opname 'add', operands (<ir.Argument '.2' of type i32>, <ir.Argument '.3' of type i32>)>]>,
 <ir.StoreInstr '.11' of type 'void', opname 'store', operands [<ir.CallInstr '.10' of type 'i32', opname 'call', operands [<ir.Function 'fib_more' of type 'i32 (i32, i32, i32)*'>, <ir.Instruction '.8' of type 'i32', opname 'sub', operands (<ir.Argument '.1' of type i32>, <ir.Constant type='i32' value=1>)>, <ir.Argument '.3' of type i32

In [48]:
call = then.instructions[2]
call??

Type:            CallInstr
String form:     %".10" = call fastcc i32 @"fib_more"(i32 %".8", i32 %".3", i32 %".9")
File:            /usr/local/miniconda3/envs/metadsl/lib/python3.7/site-packages/llvmlite/ir/instructions.py
Source:         
class CallInstr(Instruction):
    def __init__(self, parent, func, args, name='', cconv=None, tail=False,
                 fastmath=()):
        self.cconv = (func.calling_convention
                      if cconv is None and isinstance(func, Function)
                      else cconv)
        self.tail = tail
        self.fastmath = FastMathFlags(fastmath)
        self.attributes = CallInstrAttributes()

        # Fix and validate arguments
        args = list(args)
        for i in range(len(func.function_type.args)):
            arg = args[i]
            expected_type = func.function_type.args[i]
            if (isinstance(expected_type, types.MetaDataType) and
                arg.type != expected_type):
                arg = MetaDataArgument(arg)


In [49]:
call.operands

[<ir.Function 'fib_more' of type 'i32 (i32, i32, i32)*'>,
 <ir.Instruction '.8' of type 'i32', opname 'sub', operands (<ir.Argument '.1' of type i32>, <ir.Constant type='i32' value=1>)>,
 <ir.Argument '.3' of type i32>,
 <ir.Instruction '.9' of type 'i32', opname 'add', operands (<ir.Argument '.2' of type i32>, <ir.Argument '.3' of type i32>)>]

In [36]:
store = then.instructions[-2]

In [38]:
store??

Type:            StoreInstr
String form:     store i32 %".10", i32* %".7"
File:            /usr/local/miniconda3/envs/metadsl/lib/python3.7/site-packages/llvmlite/ir/instructions.py
Source:         
class StoreInstr(Instruction):
    def __init__(self, parent, val, ptr):
        super(StoreInstr, self).__init__(parent, types.VoidType(), "store",
                                         [val, ptr])

    def descr(self, buf):
        val, ptr = self.operands
        if self.align is not None:
            align = ', align %d' % (self.align)
        else:
            align = ''
        buf.append("store {0} {1}, {2} {3}{4}{5}\n".format(
            val.type,
            val.get_reference(),
            ptr.type,
            ptr.get_reference(),
            align,
            self._stringify_metadata(leading_comma=True),
            ))
Class docstring: The base class for named values.


In [40]:
alloca = store.operands[-1]

In [41]:
alloca??

Type:            AllocaInstr
String form:     %".7" = alloca i32
File:            /usr/local/miniconda3/envs/metadsl/lib/python3.7/site-packages/llvmlite/ir/instructions.py
Source:         
class AllocaInstr(Instruction):
    def __init__(self, parent, typ, count, name):
        operands = [count] if count else ()
        super(AllocaInstr, self).__init__(parent, typ.as_pointer(), "alloca",
                                          operands, name)
        self.align = None

    def descr(self, buf):
        buf.append("{0} {1}".format(self.opname, self.type.pointee))
        if self.operands:
            op, = self.operands
            buf.append(", {0} {1}".format(op.type, op.get_reference()))
        if self.align is not None:
            buf.append(", align {0}".format(self.align))
        if self.metadata:
            buf.append(self._stringify_metadata(leading_comma=True))
Class docstring: The base class for named values.


In [43]:
alloca.name

'.7'

Is this alloca used n the endif when  it is loaded?

In [53]:
endif = fib_more.blocks[4]
print(str(endif))

%"endif" = endif:
  %".23" = load i32, i32* %".7"
  ret i32 %".23"


In [57]:
load = endif.instructions[0]
load

<ir.LoadInstr '.23' of type 'i32', opname 'load', operands [<ir.AllocaInstr '.7' of type 'i32*', opname 'alloca', operands ()>]>

Yes it is. So the name doesn't matter, because we refer to the block in the other function

In [65]:
branch = fib_more.blocks[0].instructions[-1]

branch
# main_label = branch[fib_more.blocks[0]]

<ir.Branch '.25' of type 'void', opname 'br', operands [<ir.Block 'main' of type 'label'>]>

In [64]:
fib_more.blocks[1]

<ir.Block 'main' of type 'label'>

Lets look at a loop

In [66]:
!pip install numba

    100% |████████████████████████████████| 1.9MB 4.7MB/s ta 0:00:01


In [1]:
import os
os.environ['NUMBA_OPT'] = '3'

In [2]:
import numba
import numba.targets.codegen

In [3]:
mod = None

original_fn = numba.targets.codegen.CodeLibrary.add_ir_module
def add_ir_module(self, ir_module):
    global mod
    mod = ir_module
    
    return original_fn(self, ir_module)

numba.targets.codegen.CodeLibrary.add_ir_module = add_ir_module

In [4]:
@numba.njit
def loop_fn(xs):
    v = 0
    for i in xs:
        v += i
    return v

In [5]:
import numpy as np

In [6]:
loop_fn(np.arange(10))

45

In [7]:
loop_fn.inspect_cfg(loop_fn.signatures[0]).display(view=True)

'Source.gv.pdf'

In [17]:
mod

; ModuleID = "wrapper"
target triple = "x86_64-apple-darwin17.7.0"
target datalayout = "e-m:o-i64:64-f80:128-n8:16:32:64-S128"

declare i32 @"_ZN8__main__11loop_fn$244E"(i8** %".1", {i8*, i32}** %".2") 

define i8* @"_ZN7cpython8__main__11loop_fn$244E"(i8* %"py_closure", i8* %"py_args", i8* %"py_kws") 
{
entry:
  %".5" = call i32 (i8*, i8*, i64, i64, ...) @"PyArg_UnpackTuple"(i8* %"py_args", i8* bitcast ([8 x i8]* @".const.loop_fn" to i8*), i64 0, i64 0)
  %".6" = icmp eq i32 %".5", 0
  %".19" = alloca i8*
  store i8* null, i8** %".19"
  %"excinfo" = alloca {i8*, i32}*
  store {i8*, i32}* null, {i8*, i32}** %"excinfo"
  br i1 %".6", label %"entry.if", label %"entry.endif", !prof !0
entry.if:
  ret i8* null
entry.endif:
  %".10" = load i8*, i8** @"_ZN08NumbaEnv8__main__11loop_fn$244E"
  %".11" = ptrtoint i8* %".10" to i64
  %".12" = add i64 %".11", 16
  %".13" = inttoptr i64 %".12" to i8*
  %".14" = bitcast i8* %".13" to {i8*, i8*}*
  %".15" = icmp eq i8* null, %".10"
  br i1 %".15", la